# Wrangle and Analyze data

## Introduction

The dataset that we will be wrangling (and analyzing and visualizing) is the tweet archive of Twitter user @dog_rates, also known as WeRateDogs. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc. Why? Because "they're good dogs Brent." WeRateDogs has over 4 million followers and has received international media coverage.

WeRateDogs downloaded their Twitter archive and sent it to Udacity via email exclusively for you to use in this project. This archive contains basic tweet data (tweet ID, timestamp, text, etc.) for all 5000+ of their tweets as they stood on August 1, 2017. 

## Context

Goal: wrangle WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations. The Twitter archive is great, but it only contains very basic tweet information. Additional gathering, then assessing and cleaning is required for "Wow!"-worthy analyses and visualizations.

## Data Description

Enhanced Twitter Archive

The WeRateDogs Twitter archive contains basic tweet data for all 5000+ of their tweets, but not everything. One column the archive does contain though: each tweet's text, which I used to extract rating, dog name, and dog "stage" (i.e. doggo, floofer, pupper, and puppo) to make this Twitter archive "enhanced." Of the 5000+ tweets, I have filtered for tweets with ratings only (there are 2356).

Additional Data via the Twitter API

Back to the basic-ness of Twitter archives: retweet count and favorite count are two of the notable column omissions. Fortunately, this additional data can be gathered by anyone from Twitter's API. Well, "anyone" who has access to data for the 3000 most recent tweets, at least. But you, because you have the WeRateDogs Twitter archive and specifically the tweet IDs within it, can gather this data for all 5000+. And guess what? You're going to query Twitter's API to gather this valuable data.

Image Predictions File

One more cool thing: I ran every image in the WeRateDogs Twitter archive through a neural network that can classify breeds of dogs*. The results: a table full of image predictions (the top three only) alongside each tweet ID, image URL, and the image number that corresponded to the most confident prediction (numbered 1 to 4 since tweets can have up to four images).

## Key points

Key points to keep in mind when data wrangling for this project:

> We only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.

> Cleaning includes merging individual pieces of data according to the rules of tidy data.

> The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs.


## Part 1

## Project Details

Data wrangling, which consists of:
> Gathering data 

> Assessing data

> Cleaning data

In [2]:
# importing required libraries in notebook
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import tweepy
import requests
import time
import re
%matplotlib inline

## Step 1 : Gathering Data

In [3]:
# read csv file into dataframe
df_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [246]:
# Use requests library to download tsv file
url="https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsve"
response = requests.get(url)
with open('image-predictions.tsv', 'wb') as file:
    file.write(response.content)


In [4]:
df_image = pd.read_csv('image-predictions.tsv', sep='\t')

In [224]:
# consumer key and secret key generated from twitter's developer page
consumer_key = "ev3MKOfakIli3kYeL8xNiw7C7"
consumer_secret = "e4DRcI1BC6KzdF2kcZHjrMs8eCkzREeZv3RUy0Vb2KRsWknzVB"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

In [225]:
# access keys generated upon connection to twiteer through api
OAUTH_TOKEN = "789055716690722816-UJAKIsy7YGEDXr19klkCa4Lo6OmNiW6"
OAUTH_TOKEN_SECRET = "V4OoLVhxnqFSG2ZQzklFXh9m7VQO82aPGXKkzGdnaPgLa" 
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth)

In [ ]:
# accessing twitter data through api
#List of the error tweets
error_list = []
# List of tweets
df_list = []
# Calculate the time of execution
start = time.time()

# For loop which will add each available tweet json to df_list
for tweet_id in df_archive['tweet_id']:
    try:
        tweet = api.get_status(tweet_id, tweet_mode='extended',
                               wait_on_rate_limit = True, wait_on_rate_limit_notify = True)._json 
        favorites = tweet['favorite_count'] # How many favorites the tweet had
        retweets = tweet['retweet_count'] # Count of the retweet
        user_followers = tweet['user']['followers_count'] # How many followers the user had
        user_favourites = tweet['user']['favourites_count'] # How many favorites the user had
        date_time = tweet['created_at'] # The date and time of the creation
        
        df_list.append({'tweet_id': int(tweet_id),
                        'favorites': int(favorites),
                        'retweets': int(retweets),
                        'user_followers': int(user_followers),
                        'user_favourites': int(user_favourites),
                        'date_time': pd.to_datetime(date_time)})
    except Exception as e:
        print(str(tweet_id)+ " _ " + str(e))
        error_list.append(tweet_id)
# Calculate the time of excution
end = time.time()

In [ ]:
# Create DataFrames from list of dictionaries
json_tweets = pd.DataFrame(df_list, columns = ['tweet_id', 'favorites', 'retweets',
                                               'user_followers', 'user_favourites', 'date_time'])
# Save the dataFrame in file
json_tweets.to_csv('tweet_json.txt', encoding = 'utf-8', index=False)

In [5]:
# Read the saved tweet_json.txt file into a dataframe
df_tweet = pd.read_csv('tweet_json.txt', encoding = 'utf-8')

## Step 2: Assessing Data

In [11]:
# displaying information about twitter metrics data 
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2344 entries, 0 to 2343
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tweet_id         2344 non-null   int64 
 1   favorites        2344 non-null   int64 
 2   retweets         2344 non-null   int64 
 3   user_followers   2344 non-null   int64 
 4   user_favourites  2344 non-null   int64 
 5   date_time        2344 non-null   object
dtypes: int64(5), object(1)
memory usage: 110.0+ KB


In [12]:
# displaying information about twitter archived data
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [13]:
# displaying information about image output data
df_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [252]:
# analysing dataset for more understanding and finding out cleaning points
df_image

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


In [232]:
# analysing dataset for more understanding and finding out cleaning points
df_tweet

,tweet_id,favorites,retweets,user_followers,user_favourites,date_time
0,892420643555336193,38952,8636,6629272,132918,2017-08-01 16:23:56
1,892177421306343426,33352,6343,6629272,132918,2017-08-01 00:17:27
2,891815181378084864,25127,4209,6629272,132918,2017-07-31 00:18:03
3,891689557279858688,42300,8755,6629272,132918,2017-07-30 15:58:51
4,891327558926688256,40458,9517,6629272,132918,2017-07-29 16:00:24
...,...,...,...,...,...,...
2339,666049248165822465,109,40,6629803,132918,2015-11-16 00:24:50
2340,666044226329800704,301,141,6629803,132918,2015-11-16 00:04:52
2341,666033412701032449,125,44,6629803,132918,2015-11-15 23:21:54
2342,666029285002620928,131,47,6629803,132918,2015-11-15 23:05:30


In [233]:
# redundant columns in the dataset
len(df_tweet['user_favourites'].unique()),len(df_tweet['user_followers'].unique())

(2, 359)

In [283]:
# finding retweets in the dataset
df_archive[df_archive['retweeted_status_id'].notnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Canela. She attempted s...,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,8.860537e+17,1.960740e+07,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,12,10,None,None,None,None,None
36,885311592912609280,NaN,NaN,2017-07-13 01:35:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Lilly. She just paralle...,8.305833e+17,4.196984e+09,2017-02-12 01:04:29 +0000,https://twitter.com/dog_rates/status/830583320...,13,10,Lilly,None,None,None,None
68,879130579576475649,NaN,NaN,2017-06-26 00:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Emmy. She was adopted t...,8.780576e+17,4.196984e+09,2017-06-23 01:10:23 +0000,https://twitter.com/dog_rates/status/878057613...,14,10,Emmy,None,None,None,None
73,878404777348136964,NaN,NaN,2017-06-24 00:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Shadow. In an attempt to r...,8.782815e+17,4.196984e+09,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitt...",13,10,Shadow,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,746521445350707200,NaN,NaN,2016-06-25 01:52:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Shaggy. He knows exactl...,6.678667e+17,4.196984e+09,2015-11-21 00:46:50 +0000,https://twitter.com/dog_rates/status/667866724...,10,10,Shaggy,None,None,None,None
1043,743835915802583040,NaN,NaN,2016-06-17 16:01:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Extremely intelligent dog here....,6.671383e+17,4.196984e+09,2015-11-19 00:32:12 +0000,https://twitter.com/dog_rates/status/667138269...,10,10,None,None,None,None,None
1242,711998809858043904,NaN,NaN,2016-03-21 19:31:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @twitter: @dog_rates Awesome Tweet! 12/10. ...,7.119983e+17,7.832140e+05,2016-03-21 19:29:52 +0000,https://twitter.com/twitter/status/71199827977...,12,10,None,None,None,None,None
2259,667550904950915073,NaN,NaN,2015-11-20 03:51:52 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @dogratingrating: Exceptional talent. Origi...,6.675487e+17,4.296832e+09,2015-11-20 03:43:06 +0000,https://twitter.com/dogratingrating/status/667...,12,10,None,None,None,None,None


In [235]:
# value counts per name column of dataset
df_archive['name'].value_counts()

None       745
a           55
Charlie     12
Oliver      11
Cooper      11
          ... 
Jaycob       1
Jeb          1
Amber        1
Lucky        1
Hector       1
Name: name, Length: 957, dtype: int64

In [256]:
# rows with ratings that are not according to rules which could lead to erroneous information
df_archive[df_archive['rating_numerator']>20]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None
189,855860136149123072,8.558585e+17,1.361572e+07,2017-04-22 19:05:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@s8n You tried very hard to portray this good ...,NaN,NaN,NaN,NaN,666,10,None,None,None,None,None
290,838150277551247360,8.381455e+17,2.195506e+07,2017-03-04 22:12:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@markhoppus 182/10,NaN,NaN,NaN,NaN,182,10,None,None,None,None,None
313,835246439529840640,8.352460e+17,2.625958e+07,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
340,832215909146226688,NaN,NaN,2017-02-16 13:11:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: This is Logan, the Chow who liv...",7.867091e+17,4.196984e+09,2016-10-13 23:23:56 +0000,https://twitter.com/dog_rates/status/786709082...,75,10,Logan,None,None,None,None
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
695,786709082849828864,NaN,NaN,2016-10-13 23:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Logan, the Chow who lived. He solemnly...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/786709082...,75,10,Logan,None,None,None,None
763,778027034220126208,NaN,NaN,2016-09-20 00:24:34 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sophie. She's a Jubilant Bush Pupper. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/778027034...,27,10,Sophie,None,None,pupper,None
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758467244...,165,150,None,None,None,None,None


In [255]:
# rows with ratings that are not according to rules which could lead to erroneous information
df_archive[df_archive['rating_denominator'] !=10]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,2.625958e+07,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
342,832088576586297345,8.320875e+17,3.058208e+07,2017-02-16 04:45:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@docmisterio account started on 11/15/15,NaN,NaN,NaN,NaN,11,15,None,None,None,None,None
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: After so many requests, this is...",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758467244...,165,150,None,None,None,None,None
1068,740373189193256964,NaN,NaN,2016-06-08 02:41:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...","After so many requests, this is Bretagne. She ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None
1120,731156023742988288,NaN,NaN,2016-05-13 16:15:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to this unbelievably well behaved sq...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/731156023...,204,170,this,None,None,None,None
1165,722974582966214656,NaN,NaN,2016-04-21 02:25:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy 4/20 from the squad! 13/10 for all https...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/722974582...,4,20,None,None,None,None,None
1202,716439118184652801,NaN,NaN,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bluebert. He just saw that both #Final...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/716439118...,50,50,Bluebert,None,None,None,None


In [238]:
# rows with unusual names or missing names
df_archive[df_archive['name'].apply(len) <2]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
56,881536004380872706,NaN,NaN,2017-07-02 15:32:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a pupper approaching maximum borkdrive...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/881536004...,14,10,a,None,None,pupper,None
649,792913359805018113,NaN,NaN,2016-10-31 02:17:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a perfect example of someone who has t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/792913359...,13,10,a,None,None,None,None
775,776201521193218049,NaN,NaN,2016-09-14 23:30:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is O'Malley. That is how he sleeps. Doesn...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/776201521...,10,10,O,None,None,None,None
801,772581559778025472,NaN,NaN,2016-09-04 23:46:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Guys this is getting so out of hand. We only r...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/772581559...,10,10,a,None,None,None,None
1002,747885874273214464,NaN,NaN,2016-06-28 20:14:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a mighty rare blue-tailed hammer sherk...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/747885874...,8,10,a,None,None,None,None
1004,747816857231626240,NaN,NaN,2016-06-28 15:40:07 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Viewer discretion is advised. This is a terrib...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/747816857...,4,10,a,None,None,None,None
1017,746872823977771008,NaN,NaN,2016-06-26 01:08:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a carrot. We only rate dogs. Please on...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/746872823...,11,10,a,None,None,None,None
1049,743222593470234624,NaN,NaN,2016-06-15 23:24:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a very rare Great Alaskan Bush Pupper....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/743222593...,12,10,a,None,None,pupper,None
1193,717537687239008257,NaN,NaN,2016-04-06 02:21:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",People please. This is a Deadly Mediterranean ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/717537687...,11,10,a,None,None,None,None
1207,715733265223708672,NaN,NaN,2016-04-01 02:51:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a taco. We only rate dogs. Please only...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/715733265...,10,10,a,None,None,None,None


## Quality issues
which includes Completeness, Validity, Accuracy, Consistency :
> retweeted_status_id not null as we need only original tweets not retweets.

> unusual names for dogs like None,a,BO etc

> numerator and denominator in ratings that are not according to rules

> datatype for timestamp column in archive dataset 

> rcolumns in archive like retweeted_status_id ,retweeted_status_user_id etc.

> datatype for datetime column in tweet

> user_favourites,user_followers are redundant columns in tweets dataset

> missing values as number of rows not equal in all datasets

## Tidiness issues 
which inclues structural issues :
> stage  variable in four columns: doggo, floofer, pupper, puppo

> three different datasets for same data 'df_tweet' and 'df_image' and 'df_archive' 

## Cleaning
Cleaning our data is the third step in data wrangling. It is where we will fix the quality and tidiness issues that we identified in the assess step.

## How to tackle Quality issues
which includes Completeness, Validity, Accuracy, Consistency :
> remove retweeted_status_id where not null 

> mine numerator and denominator ratings that are not according to rules

> convert datatype for timestamp column in archive dataset 

> remove columns in archive like retweeted_status_id ,retweeted_status_user_id etc.

> convert datatype for datetime column in tweet

> remove user_favourites,user_followers in tweets dataset

> fill in or remove wherever necessary missing values as number of rows not equal in all datasets

## How to tackle Tidiness issues 
which inclues structural issues :
> convert stage  variable in four columns: doggo, floofer, pupper, puppo into one columns by melting.

> Merge 'df_tweet' and 'df_image' to 'df_archive' to facilitate cleaning 

In [109]:
#copying datasets to prevent changes in original dataset
clean_image = df_image.copy()
clean_tweet = df_tweet.copy()
clean_archive = df_archive.copy()

Define

Merging datasets on tweet_id using merge method

Code

In [110]:
clean_archive = clean_archive.merge(clean_tweet, on='tweet_id', how='inner')
clean_archive = clean_archive.merge(clean_image, on='tweet_id', how='inner')

Test

In [63]:
clean_archive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2068 entries, 0 to 2067
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2068 non-null   int64  
 1   in_reply_to_status_id       23 non-null     float64
 2   in_reply_to_user_id         23 non-null     float64
 3   timestamp                   2068 non-null   object 
 4   source                      2068 non-null   object 
 5   text                        2068 non-null   object 
 6   retweeted_status_id         75 non-null     float64
 7   retweeted_status_user_id    75 non-null     float64
 8   retweeted_status_timestamp  75 non-null     object 
 9   expanded_urls               2068 non-null   object 
 10  rating_numerator            2068 non-null   int64  
 11  rating_denominator          2068 non-null   int64  
 12  name                        2068 non-null   object 
 13  doggo                       2068 

Define

Drop redundant or unnecessary columns from dataset using pandas function drop

Code

In [111]:
clean_archive.drop(columns =['retweeted_status_timestamp','retweeted_status_user_id','in_reply_to_status_id','in_reply_to_user_id','user_followers','user_favourites'],inplace = True)

Test

In [112]:
clean_archive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2068 entries, 0 to 2067
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tweet_id             2068 non-null   int64  
 1   timestamp            2068 non-null   object 
 2   source               2068 non-null   object 
 3   text                 2068 non-null   object 
 4   retweeted_status_id  75 non-null     float64
 5   expanded_urls        2068 non-null   object 
 6   rating_numerator     2068 non-null   int64  
 7   rating_denominator   2068 non-null   int64  
 8   name                 2068 non-null   object 
 9   doggo                2068 non-null   object 
 10  floofer              2068 non-null   object 
 11  pupper               2068 non-null   object 
 12  puppo                2068 non-null   object 
 13  favorites            2068 non-null   int64  
 14  retweets             2068 non-null   int64  
 15  date_time            2068 non-null   o

Define

Melting stage columns into one to facilitate analysing on this column

Code

In [113]:
dog_stage = []

# Get_prediction_confidence function:
# search the first true algorithm and append it to a list with it's level of confidence
# if flase prediction_algorthm will have a value of NaN
def dog(dataframe):
    count = 0
    if dataframe['doggo'] != 'None':
        count +=1
        dog_stage.append(dataframe['doggo'])
    if dataframe['floofer'] != 'None':
        count +=1
        dog_stage.append(dataframe['floofer'])
    if dataframe['pupper'] != 'None':
        count +=1
        dog_stage.append(dataframe['pupper'])
    if dataframe['puppo'] != 'None':
        count +=1
        dog_stage.append(dataframe['puppo'])
    if count > 1:
        dog_stage.pop()
        dog_stage.pop()
        dog_stage.append('multiple')
    if count == 0:
        dog_stage.append('None')

clean_archive.apply(dog, axis=1)
clean_archive['dog_stage'] = dog_stage

In [115]:
clean_archive = clean_archive.drop(columns = ['doggo', 'floofer', 'pupper', 'puppo'])

Test

In [116]:
clean_archive['dog_stage'].value_counts()

None        1747
pupper       211
doggo         67
puppo         23
multiple      13
floofer        7
Name: dog_stage, dtype: int64

Define

Removing rows from dataset to tackle quality issues

Code

In [117]:
# Delete the retweets
clean_archive = clean_archive[pd.isnull(clean_archive.retweeted_status_id)]
clean_archive.drop(columns = 'retweeted_status_id',inplace = True)
# Delete duplicated tweet_id
clean_archive = clean_archive.drop_duplicates()

# Delete tweets with no pictures
clean_archive = clean_archive.dropna(subset = ['jpg_url'])

#Delete dog_stage duplicates
clean_archive = clean_archive.sort_values('dog_stage').drop_duplicates('tweet_id', keep = 'last')

Test

In [118]:
clean_archive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1993 entries, 0 to 781
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            1993 non-null   int64  
 1   timestamp           1993 non-null   object 
 2   source              1993 non-null   object 
 3   text                1993 non-null   object 
 4   expanded_urls       1993 non-null   object 
 5   rating_numerator    1993 non-null   int64  
 6   rating_denominator  1993 non-null   int64  
 7   name                1993 non-null   object 
 8   favorites           1993 non-null   int64  
 9   retweets            1993 non-null   int64  
 10  date_time           1993 non-null   object 
 11  jpg_url             1993 non-null   object 
 12  img_num             1993 non-null   int64  
 13  p1                  1993 non-null   object 
 14  p1_conf             1993 non-null   float64
 15  p1_dog              1993 non-null   bool   
 16  p2     

Define

Getting predictions for the dog in one column with confidence values

Code

In [119]:
# We will store the fisrt true algorithm with it's level of confidence
prediction_algorithm = []
confidence_level = []

# Get_prediction_confidence function:
# search the first true algorithm and append it to a list with it's level of confidence
# if flase prediction_algorthm will have a value of NaN
def get_prediction_confidence(dataframe):
    if dataframe['p1_dog'] == True:
        prediction_algorithm.append(dataframe['p1'])
        confidence_level.append(dataframe['p1_conf'])
    elif dataframe['p2_dog'] == True:
        prediction_algorithm.append(dataframe['p2'])
        confidence_level.append(dataframe['p2_conf'])
    elif dataframe['p3_dog'] == True:
        prediction_algorithm.append(dataframe['p3'])
        confidence_level.append(dataframe['p3_conf'])
    else:
        prediction_algorithm.append('NaN')
        confidence_level.append(0)

clean_archive.apply(get_prediction_confidence, axis=1)
clean_archive['prediction_algorithm'] = prediction_algorithm
clean_archive['confidence_level'] = confidence_level


In [120]:
# Delete the columns of image prediction information
clean_archive = clean_archive.drop(['img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], 1)

Test

In [121]:
clean_archive['prediction_algorithm'].sample() , clean_archive['confidence_level'].sample()

(568    NaN
 Name: prediction_algorithm, dtype: object,
 1905    0.367492
 Name: confidence_level, dtype: float64)

Define

Cleaning the source column 

Code

In [122]:
# Clean the content of source column
clean_archive['source'] = clean_archive['source'].apply(lambda x: re.findall(r'>(.*)<', x)[0])

Test

In [123]:
clean_archive['source'].sample()

1258    Twitter for iPhone
Name: source, dtype: object

Define

mine numerator and denominator ratings that are not according to rules

Code

In [124]:
# View all occurences where there are more than one #/# in 'text' column
text_ratings_to_fix = clean_archive[clean_archive.text.str.contains( r"(\d+\.?\d*\/\d+\.?\d*\D+\d+\.?\d*\/\d+\.?\d*)")].text

text_ratings_to_fix
for entry in text_ratings_to_fix:
    mask = clean_archive.text == entry
    column_name1 = 'rating_numerator'
    column_name2 = 'rating_denominator'
    clean_archive.loc[mask, column_name1] = re.findall(r"\d+\.?\d*\/\d+\.?\d*\D+(\d+\.?\d*)\/\d+\.?\d*", entry)
    clean_archive.loc[mask, column_name2] = 10

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Test

In [125]:
clean_archive['rating_denominator'] != 10

0       False
1382    False
1381    False
1380    False
1379    False
        ...  
325     False
749     False
454     False
12      False
781     False
Name: rating_denominator, Length: 1993, dtype: bool

Define

fixing individual rows for persistent issues in the ratings column

Code

In [126]:
# View tweets with decimals in rating in 'text' column
clean_archive[clean_archive.text.str.contains(r"(\d+\.\d*\/\d+)")]
# Set correct numerators for specific tweets
clean_archive.loc[(clean_archive['tweet_id'] == 883482846933004288) & (clean_archive['rating_numerator'] == 5), ['rating_numerator']] = 13.5
clean_archive.loc[(clean_archive['tweet_id'] == 786709082849828864) & (clean_archive['rating_numerator'] == 75), ['rating_numerator']] = 9.75
clean_archive.loc[(clean_archive['tweet_id'] == 778027034220126208) & (clean_archive['rating_numerator'] == 27), ['rating_numerator']] = 11.27
clean_archive.loc[(clean_archive['tweet_id'] == 680494726643068929) & (clean_archive['rating_numerator'] == 26), ['rating_numerator']] = 11.26

Test

In [127]:
clean_archive['rating_numerator'].astype(int)>20

0       False
1382    False
1381    False
1380    False
1379    False
        ...  
325     False
749     False
454     False
12      False
781     False
Name: rating_numerator, Length: 1993, dtype: bool

Define

filling nan values for rows having values None or 0

Code

In [128]:
clean_archive.loc[clean_archive['prediction_algorithm'] == 'NaN', 'prediction_algorithm'] = None
clean_archive.loc[clean_archive['rating_numerator'] == 'NaN', 'rating_numerator'] = 0

Test

In [129]:
sum(clean_archive['prediction_algorithm'] == None)

0

Define

converting datatypes for columns to better understand and analyse

Code

In [130]:
clean_archive['tweet_id'] = clean_archive['tweet_id'].astype(str)
clean_archive['source'] = clean_archive['source'].astype('category')
clean_archive['favorites'] = clean_archive['favorites'].astype(int)
clean_archive['retweets'] = clean_archive['retweets'].astype(int)
clean_archive['dog_stage'] = clean_archive['dog_stage'].astype('category')
clean_archive['rating_numerator'] = clean_archive['rating_numerator'].astype(float)
clean_archive['rating_denominator'] = clean_archive['rating_denominator'].astype(float)

Test

In [131]:
clean_archive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1993 entries, 0 to 781
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   tweet_id              1993 non-null   object  
 1   timestamp             1993 non-null   object  
 2   source                1993 non-null   category
 3   text                  1993 non-null   object  
 4   expanded_urls         1993 non-null   object  
 5   rating_numerator      1993 non-null   float64 
 6   rating_denominator    1993 non-null   float64 
 7   name                  1993 non-null   object  
 8   favorites             1993 non-null   int64   
 9   retweets              1993 non-null   int64   
 10  date_time             1993 non-null   object  
 11  jpg_url               1993 non-null   object  
 12  dog_stage             1993 non-null   category
 13  prediction_algorithm  1685 non-null   object  
 14  confidence_level      1993 non-null   float64 
dtypes: ca

Define

Saving dataset for use in next part of project

Code

In [132]:
# Save clean DataFrame to csv file
clean_archive.drop(clean_archive.columns[clean_archive.columns.str.contains('Unnamed',case = False)],axis = 1)
clean_archive.to_csv('twitter_archive_master.csv', encoding = 'utf-8', index=False)